<a href="https://colab.research.google.com/github/oscaraen/charlaDAQpics/blob/master/AcondicionamientoYAdquisicionCharla.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Análisis del acondicionamiento

## 1 Sensor y rango de operación

Se utilizará un sensor resistivo de platino para la temperatura (PT100) y se estimará su respuesta considerando que el comportamiento del sensor se puede modelar como 
$$R_0 = 100(1+\alpha T)$$

Cuyo valor de $\alpha = 0.0039$

Se debe estimar un rango de operación para el acondicionamiento, en este caso, vamos a obtener la respuesta desde [0 - 150]°C

El primer paso es conocer  la resistencia del sensor en cada valor del intervalo.

In [1]:
# estimación de la resistencia
r0 = 100 # resistencia a 0°c , viene dada por los parámetros del sensor
alpha = 0.0039
Temp = 150.0

Rsens = lambda T: r0 * (1+alpha * T)
print("la resistencia a %s°C es %.5f ohms"%(Temp, Rsens(150)))

la resistencia a 150.0°C es 158.50000 ohms


## 2 Convertir el parámetro resistivo Voltaje

La resistencia es difícil de medir, por tal motivo interesa convertir la variación resistiva en una variación de voltaje que es más facil de manipular eléctricamente para estimar la temperatura. Utilizamos un puente de medidas.

Conviene que el voltaje de salida sea $0V$ cuando la temperatura es de $0°C$
 <img src=https://upload.wikimedia.org/wikipedia/commons/5/5b/Wheatstone_bridge.jpg> </img>

Utilizaremos un voltaje de exitación, de $15V$ y establecemos las resistencias $R1=R2=R4 = 100 \Omega$, en el puente de medidas $R_3$ es el sensor de temperatura y $V_o$ el voltaje proporcional a la variación en la resistencia.

El voltaje de salida es de naturaleza diferencial, es decir, es la diferencia entre dos puntos de tensión diferentes a la tierra del circuito (que se asume está en el negativo de la fuente de tensión).

Para analizar el voltaje de salida, se pueden considerar dos divisores de tensión y plantear el voltaje de salida así:

\begin{equation}
v_o = v_{ex} \left[ \frac{R_{sens}}{R_{sens}+R_4} - \frac{R_{2}}{R_{2}+R_1}  \right]
\end{equation}

Asignando valores numéricos:

\begin{equation}
v_o = 15V \left[ \frac{R_{sens}}{R_{sens}+100\Omega} - \frac{100\Omega}{100\Omega+100\Omega}  \right]
\end{equation}


Apreciamos que, al aparecer dos veces el término $R_{sens}$ el puente agrega una pequeña no linealidad, que puede ser arreglada incrementando el valor de las resistencias, o puede ser calculada en el microcontrolador con la ecuación completa del puente de medidas para la estimación adecuada del valor de temperatura,

Por último, reemplazamos $R_{sens}$ con los valores límite, es decir $100\Omega$ y $158.5\Omega$, obteniendo el valor máximo y mínimo de tensión de salida



In [3]:

salidaPuente = lambda r_sens : (15 * ( r_sens / (r_sens + 100) - 0.5  ))
vo_min = salidaPuente(Rsens(0))
vo_max = salidaPuente(Rsens(150))

print("el rango de salida de tensión es: [%.2f - %.4f]V"%(vo_min, vo_max))


print("test 145 grados: ", salidaPuente(Rsens(145)) )


el rango de salida de tensión es: [0.00 - 1.6973]V
test 145 grados:  1.653186513350224


## 3 Acondicionando de tensión al rango que el microcontrolador admite

El microcontrolador que se utilizará en el ejemplo (PIC16f886) solamente permite entradas análogas en el rango de [0 - 5]V. Como se vió en el cálculo anterior, toda la variación quedó concentrada de 0V hasta 1.697V, lo cual hace indetectables pequeños cambios en la temperatura.

Es decir, la sensibilidad del puente está dada por: 

$$ M = \frac{1.69729V}{ 150°C} $$

$$M = 11.1315m V / °C $$

Si analizamos la sensibilidad del convertidor análogo a digital, que es de 10 bits en este caso, el mínimo voltaje detectable para el rango de 0 a 5 V es:

$$ v_{dmin} = \frac{5}{2^{10}} = \frac{5}{1024} = 4.8828mV $$

Para adecuar correctamente la señal y aprovechar la resolución del dispositivo de adquisición, se necesita una ganancia a la señal de salida, dada por:

$$G = \frac{5}{1.69729} = 2.94587$$

En caso de que no coincidiera el valor 0V de salida con los 0V de entrada del microcontrolador, se debe plantear una línea recta que permita mapear el rango de salida del puente al rango de entrada de tensión del convertidor análogo a digital.









In [4]:
# probando que el análisis sea correcto

acondicionamiento = lambda x: x * 2.94587

Vdaqmin = acondicionamiento(salidaPuente(Rsens(0))) #evaluendo el mínimo, temperatura 0
Vdaqmax = acondicionamiento(salidaPuente(Rsens(150))) #evaluendo el maximo, temperatura 150

print("la entrada del microcontrolador será: [%s - %.2f]V"%(Vdaqmin, Vdaqmax))



la entrada del microcontrolador será: [0.0 - 5.00]V


## 4 Función inversa 

En el microcontrolador, es necesario deshacer los pasos y las ecuaciones involucradas en el acondicionamiento de señal, esto es, la función inversa del sensor, la función inversa del puente de medidas y la función inversa del acondicionamiento, veamos:

El voltaje en el microcontrolador está dado por:

$$V_{daq} = G\times V_{o_{puente}}$$

El voltaje del puente de medidas es: 

\begin{equation}
v_o = v_{ex} \left[ \frac{R_{sens}}{R_{sens}+R_4} - \frac{R_{2}}{R_{2}+R_1}  \right]
\end{equation}

La resistencia está dada por: 
$$R_0 = 100(1+\alpha T)$$


Para encontrar $T=F(V_{daq})$ que es la función a programar tenemos que:

1. del acondicionamiento y la cuantización se tiene que:


$$V_{puente} = V_{daq} / G$$

2. Del Puente de medidas, reemplazando 

$$R_{sens} = 100 \frac{V_{puente}+7.5}{7.5-V_{puente}}$$

3. del sensor se tiene que:

$$T = \frac{R_{sens}-100}{0.39}$$


Como ultima medida se aplica la inversa de la cuantización en el ADC que es:

$$Reg = V_{daq} * \frac{1024}{5}$$



Luego, se implementa el circuito utilizando el simulador. En este caso, hay que tener en cuenta los componentes que se escogen ya que existen parámetros como desviaciones y resistencia de hilos, en el caso del sensor PT100 y las no linealidades que también se pueden simular para su posterior acondicionamiento.

Si se quiere una mayor precisión, teniendo en cuenta estos aspectos, el puente de medidas debe implementarse con resistencias de precisión (menores al 1%) como es el caso de los resistores que fabrica Vishay que tienen precisión de 0.01%

En el caso del sensor, pueden utilizarse modelos para ampliar el rango y reducir el error. Se puede reemplazar el modelo lineal (que es una aproximación) por el modelo de Kallendar Van-Dussen que es un polinomio de tercer grado y ya está demostrada su solución en función de la temperatura T.

Como último aspecto práctico, pueden haber pequeñas variaciónes de parámetros en el dispositivo físico, con lo cual es conveniente realizar un proceso de calibración estática. Este se lleva a cabo tomando múltiples medidas y entcontrando los parámetros resistivos del modelo. Así se garantiza que el error es mínimo y que la medida es de alta precisión.

Si se quiere aproximar el error se puede estimar como la raíz cuadrada del error en cada paso (error medio cuadrático)


In [0]:
finvPuente = lambda x: 100 * (x+7.5) / (-x + 7.5) # inversa del puente de medidas

In [11]:
finvPuente(1.69729) # evaluando la inversa del voltaje máximo en el puente de salidas, para encontrar el valor de Rsens

158.49990780169955

In [13]:
# Falta realizar la inversa del acondicionamiento y hallar la temperatura, ya que tenemos la resistencia

def TemperaturaRes(res):
    return (res-100)/0.39

print("la temp es: ", TemperaturaRes(finvPuente(1.69729))) # Econtramos para el voltaje de salida del puente, el valor de temperatura


la temp es:  149.9997635941014
